In [ ]:
import sys
import json
import keras
import pandas as pd
sys.path.insert(1, 'C:/Users/nconroy/Documents/EventClustering/POC/')
import utils
import numpy as np
import sklearn


#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
sess=tf.compat.v1.InteractiveSession()

import numpy as np


<b>Training set of Cscore and document features with the labels as a binary value of creating new clusters built from the gold standard test corpus</b>

In [ ]:
data_dict = utils.pd.read_pickle("models/scores_NNent_dev.pkl")

In [ ]:
print (len(data_dict))

Load and reformat the training data with scores and positive match value

In [ ]:
print (data_dict.values())

In [ ]:
data_items = data_dict.items()
data_list = list(data_items)
train_df = pd.DataFrame(data_list) 


In [ ]:
train_df = train_df.iloc[1: , :]
train_df.loc[:, 'label'] = train_df[1].map(lambda x: x[0])
train_df.loc[:, 'features'] = train_df[1].map(lambda x: x[1])

train_df = train_df.drop(1, 1)
train_df.rename(columns={0: 'ID'}, inplace=True)


In [ ]:
train_df[:40]

In [ ]:
train_df.groupby('label').count()


In [ ]:
INS = (train_df['features'].values.tolist())
OUTS = (train_df['label'].values.tolist())
def extractDigits(lst):
    return list(map(lambda el:[el], lst))

OUTS = extractDigits(OUTS)

In [ ]:
number_of_features = len(INS[1])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(INS, OUTS, test_size=0.05, shuffle=True, random_state=109) 

In [ ]:
y_test = np.array(y_test)
y_train = np.array(y_train)
X_test = np.array(X_test)
X_train = np.array(X_train)
X_train = X_train.astype(np.float)

print (y_train.shape)
print (X_train.shape)

In [ ]:
INPUTS_AMOUNT = number_of_features
HIDDEN_NODES_AMOUNT = 45
HIDDEN_NODES_AMOUNT_2 = 50
OUTPUTS_AMOUNT = 1

# define placeholder for input and output
x_ = tf.placeholder(tf.float32, shape=[None, INPUTS_AMOUNT], name="x-input")
y_ = tf.placeholder(tf.float32, shape=[None, OUTPUTS_AMOUNT], name="y-input")


<b>Define network layers and randomly initialize the weights and hidden nodes <br>
Create three hidden layers with relu-relu-sigmoid activation functions</b>

In [ ]:
# Since we're using a relu, the weights are initiated appropriately to avoid dead (-ve) neurons
### FIX WEIGHTS INITIALISATION
W = tf.Variable(tf.random_uniform([INPUTS_AMOUNT, HIDDEN_NODES_AMOUNT], -0.1, 0.1))
b = tf.Variable(tf.zeros([HIDDEN_NODES_AMOUNT]))
hidden  = tf.nn.relu(tf.matmul(x_,W) + b)

In [ ]:
### FIX WEIGHTS INITIALISATION
W1 = tf.Variable(tf.random_uniform([HIDDEN_NODES_AMOUNT, HIDDEN_NODES_AMOUNT_2], -0.1, 0.1))
b1= tf.Variable(tf.zeros([HIDDEN_NODES_AMOUNT_2]))
hidden1  = tf.nn.relu(tf.matmul(hidden,W1) + b1)

In [ ]:
### FIX WEIGHTS INITIALISATION
W2 = tf.Variable(tf.random_uniform([HIDDEN_NODES_AMOUNT_2,OUTPUTS_AMOUNT], -0.1, 0.1))
b2 = tf.Variable(tf.zeros([OUTPUTS_AMOUNT]))
hidden2 = tf.matmul(hidden1, W2) + b2
y = tf.nn.sigmoid(hidden2)

<b>Define a standard loss function and gradient optimizer for binary clf <br> <b>Load the data from the training set

In [ ]:
cost = tf.reduce_mean(( (y_ * tf.log(y)) + ((1 - y_) * tf.log(1.0 - y)) ) * -1)
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cost)


In [ ]:
X_train.dtype


In [ ]:
print (X_train)

In [ ]:
y_train.dtype

In [ ]:
sess = tf.compat.v1.InteractiveSession()  
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

In [ ]:
# Train on the input data, doesn't actually need 100000 to converge
for i in range(100000):
    sess.run(train_step, feed_dict={x_: [X_train[i%3]], y_: [y_train[i%3]]})
    if i % 2000 == 0:
        print('Output for debugging', sess.run(y, feed_dict={x_: X_train, y_: y_train}))

<b>Try making a similar network using Keras sequential model, predef initializing the graph and using different Loss functions</b>

In [ ]:
# Define a simple sequential model
import tensorflow as tf


<b>Add dense and dropout layers, use softmax activation to normalize for binary output</b>

In [ ]:
def create_model():
  model =  tf.keras.models.Sequential([
    keras.layers.Dense(25, activation='relu', input_shape=(number_of_features,)),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid'),
  ])
  print (X_train.shape)
  model.build((X_train.shape))
    
  model.compile(optimizer='adam',
                loss=tf.losses.BinaryCrossentropy(from_logits=True),
                metrics=[tf.metrics.BinaryAccuracy()])
    

  return model

In [ ]:
# Create a basic model instance
model = create_model()

In [ ]:
# Display the model's architecture
model.summary()

<b>evaluate model with cross validation on the standardized dataset</b>


In [ ]:
estimator = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=10, batch_size=2, verbose=1)
kfold =  sklearn.model_selection.StratifiedKFold(n_splits=5, shuffle=True)
results = sklearn.model_selection.cross_val_score(estimator, X_test, y_test, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


In [ ]:
import os
logdir = "ccnet_trainingNN"
graphdir = "ccnet_training/graphsNN"
checkpoint_path = "ccnet_trainingNN/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

sess = tf.compat.v1.InteractiveSession()  
init = tf.compat.v1.global_variables_initializer()
sess.run(init)
#tf.compat.v1.reset_default_graph()
cp_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir,   
                                            histogram_freq=1,    
                                                )

# Train the model with the new callback
model.fit(np.array(X_train), 
    np.array(y_train),  
    epochs=40,
    verbose=1,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback])  # Pass callback to training

#writer = tf.summary.FileWriter(graphdir, sess.graph)

In [ ]:
model_name = 'models/network_creation_Sept15_acc89.mdl'

In [ ]:
model.save_weights(model_name)

In [ ]:
model.load_weights(model_name)

In [ ]:
model.summary

In [ ]:
prediction = model.predict(X_test)

In [ ]:
for x in (list(zip (prediction, y_test))):
    print (x)